## Pre-requisits
1. create a .env file and copy env variables from the .env.example file and update the langsmith and grok keys in it

# CrewAI Multi-Agent on stock_analysis

 **STEP 1: Install Required Packages**

 Note = add "-q" to hide install logs, and you can remove "-q" if you want to print installation logs

**STEP 2: Setup & Imports**

In [10]:
import sqlite3
print(sqlite3.sqlite_version)  # Should print 3.45.0

3.37.2


In [ ]:
# Importing Crew related components
# Importing CrewAI Flow related components
# Importing CrewAI Tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import WebsiteSearchTool
# Importing AI Suite for adhoc LLM calls and Pydantic
from dotenv import load_dotenv
import os
# Load configuration from .env
load_dotenv(override=True)

True

## Set LLM properties from .env

### LiteLLM is a Python library and open-source proxy server used to simplify interactions with various Large Language Model (LLM) providers. It provides a unified interface, making it easy to call 100+ LLMs using a consistent OpenAI API format, regardless of the underlying provider. This simplifies development, allows for easy switching between models, and enables features like load balancing and cost tracking


In [12]:
from langchain_groq import ChatGroq
print("LITELLM_MODEL", os.environ.get("LITELLM_MODEL"))
llm=ChatGroq(model=os.environ.get("LITELLM_MODEL"))

LITELLM_MODEL groq/gemma2-9b-it


**STEP 3: Load Agent and Task Metadata from YAML Config**

In [13]:
import yaml

# agents_config = 'config/agents.yaml'
# tasks_config = 'config/tasks.yaml'

with open("config/tasks.yaml", "r") as f:
    tasks_config = yaml.safe_load(f)

with open("config/agents.yaml", "r") as f:
    agents_config = yaml.safe_load(f)

**STEP 4: Define Real-Time Search Tool**

In [14]:
from langchain.tools import DuckDuckGoSearchRun, Tool
search_tool = DuckDuckGoSearchRun()
news_tool = Tool(
    name="Real-time Financial News",
    func=search_tool.run,
    description="Useful to search for the latest financial news headlines related to markets."
)

**STEP 5: Define Agents from Config**

In [ ]:
# TODO: see why this code still uses OpenAI
from crewai_tools import WebsiteSearchTool
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchResults
from crewai.tools import BaseTool
from pydantic import Field

from embedchain import App
from embedchain.embedder.huggingface import HuggingFaceEmbedder
from embedchain.config import BaseEmbedderConfig
from crewai_tools import tools
from typing import Optional, Type
from pydantic import BaseModel
from langchain_community.tools import DuckDuckGoSearchRun

os.environ["OPENAI_API_KEY"] = "sk-fake-fake-fake" # huggingface need a a key populated to work
os.environ['LANGCHAIN_TRACING_V2'] = 'False'

# Use a local Hugging Face embedder with a proper config object
hf_config = BaseEmbedderConfig(model="sentence-transformers/all-MiniLM-L6-v2")
hf_embedder = HuggingFaceEmbedder(config=hf_config)

# Create an embedchain App with the local embedder and no LLM
embedchain_app = App(embedding_model=hf_embedder, llm=None)

# Create the WebsiteSearchTool with the custom embedchain app
# The WebsiteSearchTool is designed to perform a RAG 
# (Retrieval-Augmented Generation) search within the content of a website.
website_tool = WebsiteSearchTool(embedchain_app=embedchain_app)

google_search_tool = SerperDevTool()

class DuckDuckGoInput(BaseModel):
    query: str  # No special metadata here

class DuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = (
        "Search the web using DuckDuckGo. "
        "Input should be a plain string representing the search query."
    )
    args_schema: Type[BaseModel] = DuckDuckGoInput

    def _run(self, query: str) -> str:
        print(f"[DuckDuckGoTool] Received query: {query}")
        search = DuckDuckGoSearchRun()
        return search.invoke(query)

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("This tool does not support async")

duckduckgo_tool = DuckDuckGoTool()

# Test the tool initialization
print("WebsiteSearchTool initialized with Hugging Face embedder:", website_tool)
print("Google Search tool  initialized:", google_search_tool)
print("Duck Duck go tool initialized:", duckduckgo_tool)

c:\Users\noepolak1\Desktop\gen-ai-trainings\gen-ai-workshop-notebooks\genai-venv\lib\site-packages\embedchain\embedder\huggingface.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=self.config.model, model_kwargs=self.config.model_kwargs)
c:\Users\noepolak1\Desktop\gen-ai-trainings\gen-ai-workshop-notebooks\genai-venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WebsiteSearchTool initialized with Hugging Face embedder: name='Search in a specific website' description="Tool Name: Search in a specific website\nTool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search a specific website', 'type': 'str'}, 'website': {'description': 'Mandatory valid website URL you want to search on', 'type': 'str'}}\nTool Description: A tool that can be used to semantic search a query from a specific URL content." args_schema=<class 'crewai_tools.tools.website_search.website_search_tool.WebsiteSearchToolSchema'> description_updated=False cache_function=<function BaseTool.<lambda> at 0x0000022D3A56D5A0> result_as_answer=False summarize=False adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x0000022D43FD05E0>, summarize=False) config=None
Google Search tool  initialized: name='Search the internet with Serper' description="Tool Name: Search the internet with Serper\nTool Arguments: {'search_query': {'desc

In [ ]:
print(agents_config)

# print(type(news_tool))
agents = {}
for agent_cfg in agents_config['agents']:
    tools = [duckduckgo_tool] if agent_cfg.get("use_news_tool") else []
    print(agent_cfg)
    # print(tools)

    role = agent_cfg['role']
    goal=agent_cfg['goal']
    backstory=agent_cfg['backstory']
    agents[agent_cfg['id']] = Agent(role=role,goal=goal,backstory=backstory,verbose=True,llm=llm,tools=tools )
  
print(agents)

{'agents': [{'id': 'news_agent', 'role': 'NewsFetcherAgent', 'goal': 'Fetch the most recent financial news headlines', 'backstory': 'An expert financial news analyst who tracks real-time market updates.', 'use_news_tool': True}, {'id': 'insight_agent', 'role': 'InsightAnalystAgent', 'goal': 'Analyze news and extract financial insights', 'backstory': 'A financial analyst who interprets market signals and trends from raw news data.', 'use_news_tool': False}, {'id': 'risk_agent', 'role': 'RiskEvaluatorAgent', 'goal': 'Evaluate risks or volatility signals from news data', 'backstory': 'An experienced market risk analyst identifying economic and geopolitical threats.', 'use_news_tool': False}, {'id': 'advisor_agent', 'role': 'InvestmentAdvisorAgent', 'goal': 'Generate investment suggestions based on insights and risk reports', 'backstory': 'A portfolio strategist using insights and risk data to guide decisions.', 'use_news_tool': False}]}
{'id': 'news_agent', 'role': 'NewsFetcherAgent', 'go

** STEP 6: Define Tasks from Config**

In [ ]:
tasks = []
print(tasks_config)

for task_cfg in tasks_config['tasks']:
    task = Task(
        description=task_cfg['description'],
        expected_output=task_cfg['expected_output'],
        agent=agents[task_cfg['agent_id']]
    )
    tasks.append(task)

print(tasks)

{'tasks': [{'description': 'Use the Real-time Financial News tool to fetch latest market headlines', 'expected_output': 'A list of at least 5 up-to-date financial news headlines', 'agent_id': 'news_agent'}, {'description': 'Analyze the headlines and extract current market trends and financial insights', 'expected_output': 'A summary of financial insights and signals', 'agent_id': 'insight_agent'}, {'description': 'Assess potential risks and volatility based on the news headlines', 'expected_output': 'A list of risks and a risk score (1 to 10) for the current market', 'agent_id': 'risk_agent'}, {'description': 'Based on the insights and risk, recommend whether to invest, hold, or pull back', 'expected_output': 'An investment recommendation with justification', 'agent_id': 'advisor_agent'}]}
[Task(description=Use the Real-time Financial News tool to fetch latest market headlines, expected_output=A list of at least 5 up-to-date financial news headlines), Task(description=Analyze the headl

**STEP 7: Crew Execution with PARALLEL Processing (like stock_analysis example)**

In [9]:
crew = Crew(
    agents=list(agents.values()),
    tasks=tasks,
    process=Process.sequential,  # parallel execution of tasks if possible
    verbose=True
)
inputs = {
        'query': 'Can you give me analysis for company AMZN'        
    }

result = crew.kickoff(inputs=inputs)
print("\nFinal Investment Summary:\n")
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2cadf2ff-e075-4921-ab53-dd424f2e25c8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: e57fdb5b-73ff-478b-a985-a13e0050bb0b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: e57fdb5b-73ff-478b-a985-a13e0050bb0b
       Status: Executing Task...
    └── 🤖 Agent: NewsFetcherAgent
            Status: In Progress

# Agent: NewsFetcherAgent
## Task: Use the Real-time Financial News tool to fetch latest market headlines


🤖 Agent: NewsFetcherAgent
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: e57fdb5b-73ff-478b-a985-a13e0050bb0b
│      Assigned to: NewsFetcherAgent
│      Status: ✅ Completed
│   └── 🤖 Agent: NewsFetcherAgent
│           Status: ✅ Completed
├── 📋 Task: 40139d11-f3c0-4132-90cd-de662d794c4e
│      Assigned to: InsightAnalystAgent
│      Status: ✅ Completed
│   └── 🤖 Agent: InsightAnalystAgent
│           Status: ✅ Completed
├── 📋 Task: e08d005f-d67f-499b-9ed6-67ba34d6d511
│      Assigned to: RiskEvaluatorAgent
│      Status: ✅ Completed
│   └── 🤖 Agent: RiskEvaluatorAgent
│           Status: ✅ Completed
└── 📋 Task: 74fbe3ad-167a-4577-9eb3-29c199483293
       Status: Executing Task...
    └── 🤖 Agent: InvestmentAdvisorAgent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: e57fdb5b-73ff-478b-a985-a13e0050bb0b
│      Assigned to: NewsFetcherAgent
│      Status: ✅ Completed
│   └── 🤖 Agent: NewsFetcherAgent
│           Status: ✅ Completed
├── 📋 Task: 40139d11-f3c0-4132-90cd-de662d794c4e
│      Assigned to: InsightAnalystAgent
│      Status: ✅ Completed
│   └── 🤖 Agent: InsightAnalystAgent
│           Status: ✅ Completed
├── 📋 Task: e08d005f-d67f-499b-9ed6-67ba34d6d511
│      Assigned to: RiskEvaluatorAgent
│      Status: ✅ Completed
│   └── 🤖 Agent: RiskEvaluatorAgent
│           Status: ✅ Completed
└── 📋 Task: 74fbe3ad-167a-4577-9eb3-29c199483293
       Assigned to: InvestmentAdvisorAgent
       Status: ✅ Completed
    └── 🤖 Agent: InvestmentAdvisorAgent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 74fbe3ad-167a-4577-9eb3-29c199483293                                                                     │
│  Agent: InvestmentAdvisorAgent                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2cadf2ff-e075-4921-ab53-dd424f2e25c8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Investment Summary:

Based on the current market trends and risk analysis, I recommend a **hold** position.  

**Justification:**

While the US-China trade truce has injected positive sentiment into the market, leading to a rally in oil prices and broader market indices, several risks warrant caution.

* **Geopolitical Instability:**  The risk score of 6/10 highlights that geopolitical tensions, while somewhat eased by the trade agreement, remain a significant concern.  Unforeseen events could quickly disrupt the current market optimism.

* **Inflationary Pressures:**  With oil prices rising, the 7/10 risk score for inflation is a serious consideration.  Persistent inflation could erode consumer spending and corporate profits, ultimately impacting market performance.

* **Economic Slowdown:**  The 5/10 risk score for a potential economic slowdown suggests that the current market rally may not be sustainable in the long term. Rising interest rates and global uncertainty could dam